In [2]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import time
import math
from mpl_toolkits.mplot3d import Axes3D  # Needed for 3D plots
import pandas as pd  # For saving results to XLS

# Algorithm definitions (remain unchanged)
def dominates(p1, p2, criteria):
    for k in range(len(p1)):
        if criteria[k] == 'min':
            if p1[k] > p2[k]:
                return False
        else:  # 'max'
            if p1[k] < p2[k]:
                return False
    return any(
        (p1[k] < p2[k] if criteria[k] == 'min' else p1[k] > p2[k]) for k in range(len(p1))
    )

def naiveNonFilter(X, criteria):
    P = []
    number_of_comparisons = 0
    X_temp = X.copy()

    while X_temp:
        Y = X_temp[0]
        dominated_by_Y = []
        dominates_Y = False
        for j in range(1, len(X_temp)):
            number_of_comparisons += 2
            Xj = X_temp[j]
            Y_dominates_Xj = all(
                Y[k] <= Xj[k] if criteria[k] == 'min' else Y[k] >= Xj[k]
                for k in range(len(Y))
            )
            Xj_dominates_Y = all(
                Xj[k] <= Y[k] if criteria[k] == 'min' else Xj[k] >= Y[k]
                for k in range(len(Y))
            )
            if Y_dominates_Xj:
                dominated_by_Y.append(Xj)
            elif Xj_dominates_Y:
                dominates_Y = True
                break  # Y is dominated by Xj
        if not dominates_Y:
            if Y not in P:
                P.append(Y)
            X_temp.remove(Y)
            for x in dominated_by_Y:
                if x in X_temp:
                    X_temp.remove(x)
        else:
            X_temp.remove(Y)
    return P, number_of_comparisons

def naiveWithFilter(X, criteria):
    P = []
    number_of_comparisons = 0
    X_temp = X.copy()

    while X_temp:
        Y = X_temp[0]
        dominated_by_Y = []
        dominates_Y = False
        for x_j in X_temp[1:]:
            number_of_comparisons += 2
            Y_dominates_Xj = all(
                Y[k] <= x_j[k] if criteria[k] == 'min' else Y[k] >= x_j[k]
                for k in range(len(Y))
            )
            Xj_dominates_Y = all(
                x_j[k] <= Y[k] if criteria[k] == 'min' else x_j[k] >= Y[k]
                for k in range(len(Y))
            )
            if Y_dominates_Xj:
                dominated_by_Y.append(x_j)
            elif Xj_dominates_Y:
                dominates_Y = True
                break  # Restart with new Y
        if dominates_Y:
            X_temp.remove(Y)
            continue  # Restart loop with new Y
        else:
            if Y not in P:
                P.append(Y)
            X_temp.remove(Y)
            for x in dominated_by_Y:
                if x in X_temp:
                    X_temp.remove(x)
    return P, number_of_comparisons

def distance(p1, p2):
    return math.sqrt(sum((p1[k] - p2[k]) ** 2 for k in range(len(p1))))

def calculate_ideal_point(X, criteria):
    ideal = []
    for k in range(len(X[0])):
        if criteria[k] == 'min':
            ideal.append(min(point[k] for point in X))
        else:
            ideal.append(max(point[k] for point in X))
    return tuple(ideal)

def naiveIdealPoint(X, criteria):
    ideal = calculate_ideal_point(X, criteria)

    distances = [(distance(p, ideal), idx) for idx, p in enumerate(X)]
    distances.sort(key=lambda x: x[0])

    sorted_indices = [index for _, index in distances]

    P = []
    number_of_comparisons = 0
    X_temp = X.copy()
    m = 0

    while X_temp and m < len(sorted_indices):
        current_p = X[sorted_indices[m]]
        if current_p in X_temp:
            P.append(current_p)
            X_temp.remove(current_p)

            dominated_p = []
            for p in X_temp:
                number_of_comparisons += 1
                if dominates(current_p, p, criteria):
                    dominated_p.append(p)

            for dp in dominated_p:
                if dp in X_temp:
                    X_temp.remove(dp)

        m += 1

    return P, number_of_comparisons

# Application definition
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Finding Non-dominated Points")
        self.geometry("1200x800")
        self.create_widgets()

    def create_widgets(self):
        # Main frame
        main_frame = ttk.Frame(self)
        main_frame.pack(fill="both", expand=True)

        # Configure grid
        main_frame.columnconfigure(0, weight=1, minsize=400)  # Left column
        main_frame.columnconfigure(1, weight=1, minsize=600)  # Right column
        main_frame.rowconfigure(0, weight=1)

        # Left frame for inputs
        left_frame = ttk.Frame(main_frame)
        left_frame.grid(row=0, column=0, sticky="nsew")
        left_frame.columnconfigure(0, weight=1)
        left_frame.rowconfigure(1, weight=1)  # Make criteria area expand

        # Right frame for results
        right_frame = ttk.Frame(main_frame)
        right_frame.grid(row=0, column=1, sticky="nsew")
        right_frame.columnconfigure(0, weight=1)
        right_frame.rowconfigure(0, weight=1)

        # Input configuration frame within left_frame
        input_frame = ttk.LabelFrame(left_frame, text="Input Configuration")
        input_frame.grid(row=0, column=0, sticky="nsew", padx=10, pady=5)
        input_frame.columnconfigure(1, weight=1)
        self.input_frame = input_frame

        # Number of points
        ttk.Label(input_frame, text="Number of points:").grid(row=0, column=0, sticky="w", padx=5, pady=5)
        self.num_points_var = tk.IntVar(value=50)
        ttk.Entry(input_frame, textvariable=self.num_points_var).grid(row=0, column=1, padx=5, pady=5, sticky="ew")

        # Number of criteria
        ttk.Label(input_frame, text="Number of criteria:").grid(row=1, column=0, sticky="w", padx=5, pady=5)
        self.num_criteria_var = tk.IntVar(value=2)
        ttk.Entry(input_frame, textvariable=self.num_criteria_var).grid(row=1, column=1, padx=5, pady=5, sticky="ew")
        ttk.Button(input_frame, text="Update Criteria", command=self.update_criteria_inputs).grid(row=1, column=2, padx=5, pady=5)

        # Distribution selection
        ttk.Label(input_frame, text="Select distribution:").grid(row=2, column=0, sticky="w", padx=5, pady=5)
        self.distribution_var = tk.StringVar(value="Uniform")
        distributions = ["Uniform", "Normal", "Exponential", "Beta"]
        self.distribution_menu = ttk.OptionMenu(input_frame, self.distribution_var, "Uniform", *distributions, command=self.update_distribution_params)
        self.distribution_menu.grid(row=2, column=1, padx=5, pady=5, sticky="ew")

        # Frame for distribution parameters
        self.distribution_params_frame = ttk.Frame(input_frame)
        self.distribution_params_frame.grid(row=3, column=0, columnspan=3, sticky="ew", padx=5, pady=5)
        self.distribution_params_frame.columnconfigure(1, weight=1)
        self.distribution_params = {}  # Dictionary to hold parameter variables

        # Initialize distribution parameters
        self.update_distribution_params()

        # Canvas and scrollbar for criteria inputs
        criteria_frame = ttk.Frame(left_frame)
        criteria_frame.grid(row=1, column=0, sticky="nsew", padx=5, pady=5)
        criteria_frame.columnconfigure(0, weight=1)
        criteria_frame.rowconfigure(0, weight=1)

        # Create a canvas inside criteria_frame
        criteria_canvas = tk.Canvas(criteria_frame)
        criteria_canvas.grid(row=0, column=0, sticky="nsew")

        # Add a scrollbar to the canvas
        criteria_scrollbar = ttk.Scrollbar(criteria_frame, orient="vertical", command=criteria_canvas.yview)
        criteria_scrollbar.grid(row=0, column=1, sticky="ns")

        criteria_canvas.configure(yscrollcommand=criteria_scrollbar.set)

        # Create a frame inside the canvas to hold criteria inputs
        self.criteria_container = ttk.Frame(criteria_canvas)

        # Add the criteria_container to the canvas
        self.canvas_window = criteria_canvas.create_window((0, 0), window=self.criteria_container, anchor="nw")

        # Bind the criteria_container to configure the scroll region
        self.criteria_container.bind("<Configure>", lambda e: criteria_canvas.configure(scrollregion=criteria_canvas.bbox("all")))

        # Bind the canvas to adjust the width of the criteria_container
        criteria_canvas.bind("<Configure>", self.on_canvas_configure)

        # Save references to the canvas and criteria_frame
        self.criteria_canvas = criteria_canvas
        self.criteria_frame = criteria_frame

        # Initialize criteria inputs
        self.criteria_frames = []
        self.criteria_ranges = []
        self.criteria_minmax = []
        self.update_criteria_inputs()

        # Buttons frame
        buttons_frame = ttk.Frame(input_frame)
        buttons_frame.grid(row=4, column=0, columnspan=3, sticky="ew", padx=5, pady=5)

        self.generate_button = ttk.Button(buttons_frame, text="Generate Data", command=self.generate_data)
        self.generate_button.pack(side="left", padx=5, pady=5)
        self.run_button = ttk.Button(buttons_frame, text="Run Algorithms", command=self.run_algorithms)
        self.run_button.pack(side="left", padx=5, pady=5)
        self.save_button = ttk.Button(buttons_frame, text="Save Results to XLS", command=self.save_to_excel)
        self.save_button.pack(side="left", padx=5, pady=5)

        # Algorithm selection frame
        algo_frame = ttk.LabelFrame(left_frame, text="Algorithms")
        algo_frame.grid(row=2, column=0, sticky="ew", padx=10, pady=5)
        algo_frame.columnconfigure(0, weight=1)

        self.algo_vars = {}
        self.algo_names = ["Naive Non-Filter", "Naive With Filter", "Naive Ideal Point"]
        for i, name in enumerate(self.algo_names):
            var = tk.BooleanVar(value=True)
            self.algo_vars[name] = var
            ttk.Checkbutton(algo_frame, text=name, variable=var).pack(anchor="w", padx=5, pady=2)

        # Results frame within right_frame
        result_frame = ttk.Frame(right_frame)
        result_frame.pack(fill="both", expand=True, padx=10, pady=5)
        result_frame.columnconfigure(0, weight=1)
        result_frame.rowconfigure(1, weight=1)

        # Result text
        self.result_text = tk.Text(result_frame, height=10)
        self.result_text.grid(row=0, column=0, sticky="ew", padx=5, pady=5)

        # Notebook for plot and table
        self.notebook = ttk.Notebook(result_frame)
        self.notebook.grid(row=1, column=0, sticky="nsew")

        # Plot frame
        self.plot_frame = ttk.Frame(self.notebook)
        self.notebook.add(self.plot_frame, text='Plot')

        # Plot
        self.figure = plt.Figure(figsize=(6, 5))
        self.canvas = FigureCanvasTkAgg(self.figure, master=self.plot_frame)
        self.canvas.get_tk_widget().pack(fill="both", expand=True)

        # Table frame
        self.table_frame = ttk.Frame(self.notebook)
        self.notebook.add(self.table_frame, text='Data Table')

        # Data table
        self.data_table = ttk.Treeview(self.table_frame, show='headings')
        self.data_table.pack(side='left', fill='both', expand=True)

        # Vertical scrollbar for data table
        self.table_v_scrollbar = ttk.Scrollbar(self.table_frame, orient="vertical", command=self.data_table.yview)
        self.table_v_scrollbar.pack(side='right', fill='y')

        # Horizontal scrollbar for data table
        self.table_h_scrollbar = ttk.Scrollbar(self.table_frame, orient="horizontal", command=self.data_table.xview)
        self.table_h_scrollbar.pack(side='bottom', fill='x')

        # Configure data table scrollbars
        self.data_table.configure(yscrollcommand=self.table_v_scrollbar.set, xscrollcommand=self.table_h_scrollbar.set)

    def on_canvas_configure(self, event):
        # Adjust the width of the inner frame to fill the canvas
        canvas_width = event.width
        self.criteria_canvas.itemconfig(self.canvas_window, width=canvas_width)

    def update_distribution_params(self, *args):
        # Clear previous parameters
        for widget in self.distribution_params_frame.winfo_children():
            widget.destroy()
        self.distribution_params.clear()

        distribution = self.distribution_var.get()
        if distribution == "Normal":
            # Mean and Standard Deviation inputs
            ttk.Label(self.distribution_params_frame, text="Mean:").grid(row=0, column=0, sticky="w", padx=5, pady=2)
            mean_var = tk.DoubleVar(value=0)
            ttk.Entry(self.distribution_params_frame, textvariable=mean_var).grid(row=0, column=1, sticky="ew", padx=5, pady=2)
            self.distribution_params['mean'] = mean_var

            ttk.Label(self.distribution_params_frame, text="Standard Deviation:").grid(row=1, column=0, sticky="w", padx=5, pady=2)
            std_dev_var = tk.DoubleVar(value=1)
            ttk.Entry(self.distribution_params_frame, textvariable=std_dev_var).grid(row=1, column=1, sticky="ew", padx=5, pady=2)
            self.distribution_params['std_dev'] = std_dev_var

        elif distribution == "Beta":
            # Alpha and Beta parameters
            ttk.Label(self.distribution_params_frame, text="Alpha (a):").grid(row=0, column=0, sticky="w", padx=5, pady=2)
            alpha_var = tk.DoubleVar(value=2)
            ttk.Entry(self.distribution_params_frame, textvariable=alpha_var).grid(row=0, column=1, sticky="ew", padx=5, pady=2)
            self.distribution_params['alpha'] = alpha_var

            ttk.Label(self.distribution_params_frame, text="Beta (b):").grid(row=1, column=0, sticky="w", padx=5, pady=2)
            beta_var = tk.DoubleVar(value=5)
            ttk.Entry(self.distribution_params_frame, textvariable=beta_var).grid(row=1, column=1, sticky="ew", padx=5, pady=2)
            self.distribution_params['beta'] = beta_var

        elif distribution == "Exponential":
            # Scale (lambda) parameter
            ttk.Label(self.distribution_params_frame, text="Scale (λ):").grid(row=0, column=0, sticky="w", padx=5, pady=2)
            scale_var = tk.DoubleVar(value=1)
            ttk.Entry(self.distribution_params_frame, textvariable=scale_var).grid(row=0, column=1, sticky="ew", padx=5, pady=2)
            self.distribution_params['scale'] = scale_var

    def update_criteria_inputs(self):
        # Remove previous criteria frames
        for frame in self.criteria_frames:
            frame.destroy()
        self.criteria_frames.clear()
        self.criteria_ranges.clear()
        self.criteria_minmax.clear()

        try:
            num_criteria = self.num_criteria_var.get()
            if num_criteria < 1:
                raise ValueError
        except ValueError:
            messagebox.showwarning("Invalid number of criteria", "Please enter a valid number of criteria (integer greater than 0).")
            return

        # Now, create criteria inputs inside self.criteria_container
        for i in range(num_criteria):
            frame = ttk.Frame(self.criteria_container)
            frame.pack(fill="x", padx=5, pady=2)
            self.criteria_frames.append(frame)

            # Configure grid weights to make widgets expand
            frame.columnconfigure(0, weight=0)
            frame.columnconfigure(1, weight=1)
            frame.columnconfigure(2, weight=1)
            frame.columnconfigure(3, weight=0)
            frame.columnconfigure(4, weight=1)

            ttk.Label(frame, text=f"Criterion {i+1} range (min, max):").grid(row=0, column=0, padx=5, pady=5, sticky="w")
            min_var = tk.DoubleVar(value=0)
            max_var = tk.DoubleVar(value=10)
            ttk.Entry(frame, textvariable=min_var).grid(row=0, column=1, padx=5, pady=5, sticky="ew")
            ttk.Entry(frame, textvariable=max_var).grid(row=0, column=2, padx=5, pady=5, sticky="ew")
            self.criteria_ranges.append((min_var, max_var))

            ttk.Label(frame, text=f"Criterion {i+1} (Min/Max):").grid(row=0, column=3, padx=5, pady=5, sticky="w")
            minmax_var = tk.StringVar(value="Minimize")
            ttk.OptionMenu(frame, minmax_var, "Minimize", "Minimize", "Maximize").grid(row=0, column=4, padx=5, pady=5, sticky="ew")
            self.criteria_minmax.append(minmax_var)

    def generate_data(self):
        num_points = self.num_points_var.get()
        num_criteria = self.num_criteria_var.get()

        self.X = []
        distribution = self.distribution_var.get()

        # Get distribution parameters
        params = self.distribution_params

        # Error handling for distribution parameters
        try:
            if distribution == "Normal":
                mean = params['mean'].get()
                std_dev = params['std_dev'].get()
                if std_dev <= 0:
                    raise ValueError("Standard deviation must be positive.")
            elif distribution == "Beta":
                alpha = params['alpha'].get()
                beta = params['beta'].get()
                if alpha <= 0 or beta <= 0:
                    raise ValueError("Alpha and Beta must be positive.")
            elif distribution == "Exponential":
                scale = params['scale'].get()
                if scale <= 0:
                    raise ValueError("Scale must be positive.")
        except Exception as e:
            messagebox.showerror("Invalid Parameters", str(e))
            return

        for _ in range(num_points):
            point = []
            for i in range(num_criteria):
                min_val = self.criteria_ranges[i][0].get()
                max_val = self.criteria_ranges[i][1].get()

                if distribution == "Uniform":
                    val = np.random.uniform(min_val, max_val)
                elif distribution == "Normal":
                    val = np.random.normal(mean, std_dev)
                    val = max(min(val, max_val), min_val)  # Clamp to min and max
                elif distribution == "Exponential":
                    val = np.random.exponential(scale) + min_val
                    val = min(val, max_val)  # Clamp to max_val
                elif distribution == "Beta":
                    val = np.random.beta(alpha, beta) * (max_val - min_val) + min_val
                else:
                    val = np.random.uniform(min_val, max_val)
                point.append(val)
            self.X.append(tuple(point))

        self.result_text.insert(tk.END, f"Generated {num_points} points with {num_criteria} criteria.\n")
        self.plot_data()

    def run_algorithms(self):
        selected_algos = [name for name in self.algo_names if self.algo_vars[name].get()]
        if not selected_algos:
            messagebox.showwarning("No algorithm selected", "Please select at least one algorithm.")
            return

        if not hasattr(self, 'X') or not self.X:
            messagebox.showwarning("No data", "Please generate data first.")
            return

        # Get criteria
        criteria = []
        for minmax_var in self.criteria_minmax:
            if minmax_var.get() == "Minimize":
                criteria.append('min')
            else:
                criteria.append('max')

        # Clear previous results
        self.result_text.delete(1.0, tk.END)

        plot_available = len(self.X[0]) <= 3

        if plot_available:
            self.figure.clear()
            if len(self.X[0]) == 2:
                self.ax = self.figure.add_subplot(111)
                all_xs = [p[0] for p in self.X]
                all_ys = [p[1] for p in self.X]
                self.ax.scatter(all_xs, all_ys, color='gray', alpha=0.5, label='All Points')
                self.ax.set_xlabel('Criterion 1')
                self.ax.set_ylabel('Criterion 2')
            elif len(self.X[0]) == 3:
                self.ax = self.figure.add_subplot(111, projection='3d')
                all_xs = [p[0] for p in self.X]
                all_ys = [p[1] for p in self.X]
                all_zs = [p[2] for p in self.X]
                self.ax.scatter(all_xs, all_ys, all_zs, color='gray', alpha=0.5, label='All Points')
                self.ax.set_xlabel('Criterion 1')
                self.ax.set_ylabel('Criterion 2')
                self.ax.set_zlabel('Criterion 3')
        else:
            self.figure.clear()
            messagebox.showinfo("Visualization", "Visualization is available only for 2 or 3 criteria.")

        colors = {'Naive Non-Filter': 'red', 'Naive With Filter': 'blue', 'Naive Ideal Point': 'green'}
        markers = {'Naive Non-Filter': 'o', 'Naive With Filter': '^', 'Naive Ideal Point': 's'}

        # Prepare data table
        for item in self.data_table.get_children():
            self.data_table.delete(item)
        self.data_table['columns'] = ()

        criteria_columns = [f'C{i+1}' for i in range(len(self.X[0]))]
        columns = ['ID'] + criteria_columns + selected_algos
        self.data_table['columns'] = columns

        # Remove any existing column configurations
        for col in self.data_table['columns']:
            self.data_table.heading(col, text='')
            self.data_table.column(col, width=0)

        # Configure columns with fixed width and disable stretching
        column_width = 80  # Adjust this value as needed
        for col in columns:
            self.data_table.heading(col, text=col)
            self.data_table.column(col, width=column_width, anchor='center', stretch=False)

        self.results_data = []  # For saving results
        non_dominated_indices = {}

        for algo_name in selected_algos:
            start_time = time.time()
            if algo_name == "Naive Non-Filter":
                P, no = naiveNonFilter(self.X.copy(), criteria)
            elif algo_name == "Naive With Filter":
                P, no = naiveWithFilter(self.X.copy(), criteria)
            elif algo_name == "Naive Ideal Point":
                P, no = naiveIdealPoint(self.X.copy(), criteria)
            else:
                continue
            end_time = time.time()
            elapsed_time = end_time - start_time

            self.result_text.insert(tk.END, f"{algo_name}:\n")
            self.result_text.insert(tk.END, f"Number of comparisons: {no}\n")
            self.result_text.insert(tk.END, f"Execution time: {elapsed_time:.6f} seconds\n")
            self.result_text.insert(tk.END, f"Number of non-dominated points: {len(P)}\n\n")

            # Map points to indices
            indices = set()
            for p in P:
                for idx, original_p in enumerate(self.X):
                    if p == original_p:
                        indices.add(idx)
                        break
            non_dominated_indices[algo_name] = indices

            # Plot non-dominated points
            if plot_available:
                if len(self.X[0]) == 2:
                    xs = [p[0] for p in P]
                    ys = [p[1] for p in P]
                    self.ax.scatter(xs, ys, color=colors.get(algo_name, 'black'), marker=markers.get(algo_name, 'o'), label=f"{algo_name}")
                elif len(self.X[0]) == 3:
                    xs = [p[0] for p in P]
                    ys = [p[1] for p in P]
                    zs = [p[2] for p in P]
                    self.ax.scatter(xs, ys, zs, color=colors.get(algo_name, 'black'), marker=markers.get(algo_name, 'o'), label=f"{algo_name}")

        # Update data table
        for idx, point in enumerate(self.X):
            row = [idx] + [f"{val:.2f}" for val in point]
            result_row = {'ID': idx}
            for i, val in enumerate(point):
                result_row[f'C{i+1}'] = val
            for algo_name in selected_algos:
                status = 'Non-dominated' if idx in non_dominated_indices[algo_name] else 'Dominated'
                row.append(status)
                result_row[algo_name] = status
            self.data_table.insert('', 'end', values=row)
            self.results_data.append(result_row)

        if plot_available and len(selected_algos) > 0:
            self.ax.legend()
            self.canvas.draw()

        # Force the left frame to maintain its width
        self.update_idletasks()
        self.criteria_canvas.configure(scrollregion=self.criteria_canvas.bbox("all"))
        self.criteria_canvas.itemconfig(self.canvas_window, width=self.criteria_canvas.winfo_width())

    def plot_data(self):
        if len(self.X[0]) > 3:
            self.figure.clear()
            messagebox.showinfo("Visualization", "Visualization is available only for 2 or 3 criteria.")
            return
        self.figure.clear()
        if len(self.X[0]) == 2:
            self.ax = self.figure.add_subplot(111)
            xs = [p[0] for p in self.X]
            ys = [p[1] for p in self.X]
            self.ax.scatter(xs, ys, color='gray', alpha=0.5, label='All Points')
            self.ax.set_xlabel('Criterion 1')
            self.ax.set_ylabel('Criterion 2')
        elif len(self.X[0]) == 3:
            self.ax = self.figure.add_subplot(111, projection='3d')
            xs = [p[0] for p in self.X]
            ys = [p[1] for p in self.X]
            zs = [p[2] for p in self.X]
            self.ax.scatter(xs, ys, zs, color='gray', alpha=0.5, label='All Points')
            self.ax.set_xlabel('Criterion 1')
            self.ax.set_ylabel('Criterion 2')
            self.ax.set_zlabel('Criterion 3')
        self.canvas.draw()

    def save_to_excel(self):
        if not hasattr(self, 'results_data') or not self.results_data:
            messagebox.showwarning("No results", "Please run the algorithms first.")
            return

        file_path = filedialog.asksaveasfilename(defaultextension='.xlsx', filetypes=[('Excel files', '*.xlsx'), ('All files', '*.*')])
        if not file_path:
            return

        df = pd.DataFrame(self.results_data)
        try:
            df.to_excel(file_path, index=False)
            messagebox.showinfo("Saved", f"Results have been saved to {file_path}")
        except Exception as e:
            messagebox.showerror("Save Error", f"Could not save the file: {e}")

if __name__ == "__main__":
    app = App()
    app.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\Python39\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\lszys\AppData\Local\Temp\ipykernel_1628\3817086383.py", line 569, in run_algorithms
    self.ax.legend()
  File "C:\Program Files\Python39\lib\tkinter\__init__.py", line 2354, in __getattr__
    return getattr(self.tk, attr)
AttributeError: '_tkinter.tkapp' object has no attribute 'ax'
